<a href="https://colab.research.google.com/github/yermakhan-s/Amazon-Product-Review-Sentiment-Analysis-BERT-model/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import numpy as np
import os
import pandas as pd
import sys
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, BertModel, BertForSequenceClassification

In [ ]:
!pip3 install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
ratings = []
texts = []

for d in dataset['full']:
    ratings.append(d['rating'])
    texts.append(d['text'])

In [ ]:
for i in range(len(ratings)):
    if ratings[i] == 1 or ratings[i] == 2:
        ratings[i] = 1
    elif ratings[i] == 3:
        ratings[i] = 2
    elif ratings[i] == 4 or ratings[i] == 5:
        ratings[i] = 3

In [ ]:
counts = {1: 0, 2: 0, 3: 0}

# Count occurrences of each value
for value in ratings:
    counts[value] += 1

print("Number of 0s:", counts[1])
print("Number of 1s:", counts[2])
print("Number of 2s:", counts[3])

Number of 0s: 145114
Number of 1s: 56307
Number of 2s: 500107


In [ ]:
unique_values = set(ratings)

# Convert the set back to a list if needed
unique_list = list(unique_values)

print(unique_list)

NameError: name 'ratings' is not defined

In [ ]:
import pandas as pd


class_counts = {1: 0, 2: 0, 3: 0}

new_ratings = []
new_texts = []
# Iterate over ratings and texts simultaneously
for rating, text in zip(ratings, texts):
    if class_counts[rating] < 10000:
        new_ratings.append(rating)
        new_texts.append(text)
        class_counts[rating] += 1
    # If we have collected 25000 examples from each class, stop
    if all(count == 10000 for count in class_counts.values()):
        break


In [ ]:
data = {'text': new_texts, 'rating': new_ratings}
df = pd.DataFrame(data)
df.head()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = len(df['rating'].unique()), # number of unique labels for our multi-class classification problem
    output_attentions = False,
    output_hidden_states = False,
)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
class ReviewsDataset(Dataset):
    def __init__(self, df, max_length=512):
        self.df = df
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # input=review, label=stars
        review = self.df.loc[idx, 'text']
        # labels are 0-indexed
        label = int(self.df.loc[idx, 'rating']) - 1

        encoded = self.tokenizer(
            review,                      # review to encode
            add_special_tokens=True,
            max_length=self.max_length,  # Truncate all segments to max_length
            padding='max_length',        # pad all reviews with the [PAD] token to the max_length
            return_attention_mask=True,  # Construct attention masks.
            truncation=True
        )

        input_ids = encoded['input_ids']
        attn_mask = encoded['attention_mask']

        return {
            'input_ids': torch.tensor(input_ids),
            'attn_mask': torch.tensor(attn_mask),
            'label': torch.tensor(label)
        }

In [ ]:
train_dataset, test_dataset = train_test_split(df, test_size=0.2, random_state=1)

In [ ]:
MAX_LEN = 256
TEST_SIZE = 0.2
VAL_SIZE = 0.125
TRAIN_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32

CHECKPOINT_FILE = 'checkpoint.dat'
CHECKPOINT_FOLDER = '/content/drive/MyDrive/Colab Notebooks/'
EPOCHS = 1
LEARNING_RATE = 2e-05
PROJECT_FOLDER = '/content/drive/MyDrive/Colab Notebooks/'
MODEL_FOLDER = 'Model'
SAVE_EVERY = 100
NUM_WORKERS = 4

In [ ]:
train_dataset, test_dataset = train_test_split(df, test_size=TEST_SIZE, random_state=1)
train_dataset, val_dataset = train_test_split(train_dataset, test_size=VAL_SIZE, random_state=1)

train_dataset = train_dataset.reset_index(drop=True)
val_dataset = val_dataset.reset_index(drop=True)
test_dataset = test_dataset.reset_index(drop=True)

train_set = ReviewsDataset(train_dataset, MAX_LEN)
val_set = ReviewsDataset(val_dataset, MAX_LEN)
test_set = ReviewsDataset(test_dataset, MAX_LEN)

print("# of samples in train set: {}".format(len(train_set)))
print("# of samples in val set: {}".format(len(val_set)))
print("# of samples in test set: {}".format(len(test_set)))

# of samples in train set: 21000
# of samples in val set: 3000
# of samples in test set: 6000


In [ ]:
train_params = {
                'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': NUM_WORKERS
                }
val_params = train_params

test_params = {
                'batch_size': TEST_BATCH_SIZE,
                'shuffle': False,
                'num_workers': NUM_WORKERS
              }

train_loader = DataLoader(train_set, **train_params)
val_loader = DataLoader(val_set, **val_params)
test_loader = DataLoader(test_set, **test_params)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# For weighted Cross Entropy Loss
# Penalize errors higher if they come from a class with lower frequency
star_groups = df.groupby('rating')
print(star_groups.head())
star_distribution = []
for i in range(len(df['rating'].unique())):
    star_distribution.append(len(star_groups.groups[i+1])/len(df))

star_distribution = torch.tensor(star_distribution, dtype=torch.float32)

# V3
weights = 1.0 / star_distribution
weights = weights / weights.sum()

# V4
# weights = 1.0 - star_distribution

print('{:<20}: {}'.format('Star distribution', star_distribution.tolist()))
print('{:<20}: {}'.format('Weights', weights.tolist()))

                                                 text  rating
0   This spray is really nice. It smells really go...       3
1   This product does what I need it to do, I just...       3
2                           Smells good, feels great!       3
3                                      Felt synthetic       1
4                                             Love it       3
5   The polish was quiet thick and did not apply s...       3
7   These were lightweight and soft but much too s...       2
10  I was very disappointed when I got this facial...       2
12  I try to get Keratin treatments every 3 months...       2
15  When I saw this, I was thrilled to be able to ...       2
22  I don't see the fuss with this toothbrush. As ...       2
27  nothing special  unfortunately i waited too lo...       1
29  I think I need to stick with my 5 blade razor....       1
30  A total waste of money.  I get better results ...       1
32  This halo hair extension is simply put, garbag...       1
Star dis

In [ ]:
# Define the optimizer
loss_function = torch.nn.CrossEntropyLoss(weight=weights.to(device), reduction='mean')
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Define the accuracy function
def calculate_accuracy(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [ ]:
# For validation
def validate(model, data_loader):
    model.eval()
    n_correct = 0
    nb_test_steps = 0
    nb_test_examples = 0
    test_loss = 0
    y_pred = []
    y_true = []

    with torch.no_grad():
        for _, data in enumerate(data_loader, 0):
            input_ids = data['input_ids'].to(device)
            mask = data['attn_mask'].to(device)
            labels = data['label'].to(device)

            outputs = model(input_ids, mask)
            loss = loss_function(outputs[0], labels)
            test_loss += loss.item()

            # gets labels with highest probabilities and their corresponding indices
            big_val, big_idx = torch.max(outputs[0].data, dim=1)
            n_correct += calculate_accuracy(big_idx, labels)

            preds = (big_idx + 1).cpu().tolist()
            gold = (labels + 1).cpu().tolist()
            y_pred.extend(preds)
            y_true.extend(gold)

            nb_test_steps += 1
            nb_test_examples += labels.size(0)

    epoch_loss = test_loss/nb_test_steps
    epoch_accu = (n_correct*100)/nb_test_examples
    print(f"Validation Loss: {epoch_loss}")
    print(f"Validation Accuracy: {epoch_accu}\n")

    return y_true, y_pred, epoch_accu

In [ ]:
# Training loop
def train(epoch):
    # number of batches run by model
    nb_tr_steps = 0
    # number of training examples run by model
    nb_tr_examples = 0
    # number of examples classified correctly by model
    n_correct = 0
    tr_loss = 0
    model.train()

    for batch, data in enumerate(train_loader):
        input_ids = data['input_ids'].to(device)
        mask = data['attn_mask'].to(device)
        labels = data['label'].to(device)

        outputs = model(input_ids, mask)
        loss = loss_function(outputs[0], labels)
        tr_loss += loss.item()

        # gets labels with highest probabilities and their corresponding indices
        big_val, big_idx = torch.max(outputs[0].data, dim=1)
        n_correct += calculate_accuracy(big_idx, labels)

        nb_tr_steps += 1
        nb_tr_examples+=labels.size(0)

        if batch % SAVE_EVERY == 0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print("Batch {} of epoch {} complete.".format(batch, epoch+1))
            print(f"Training Loss: {loss_step}   Training Accuracy: {accu_step}")

            if not os.path.exists(CHECKPOINT_FOLDER):
              os.makedirs(CHECKPOINT_FOLDER)

            # Since a single epoch could take well over hours, we regularly save the model even during evaluation of training accuracy.
            torch.save(model.state_dict(), os.path.join(PROJECT_FOLDER, CHECKPOINT_FOLDER, CHECKPOINT_FILE))
            print("Saving checkpoint at", os.path.join(PROJECT_FOLDER, CHECKPOINT_FOLDER, CHECKPOINT_FILE))

        optimizer.zero_grad()
        loss.backward()
        # When using GPU
        optimizer.step()

    print('\n*****\n')
    print(f'The Total Accuracy for Epoch {epoch+1}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss: {epoch_loss}")
    print(f"Training Accuracy: {epoch_accu}\n")

    # Evaluate model after training it on this epoch
    validate(model, val_loader)

    torch.save(model.state_dict(), os.path.join(PROJECT_FOLDER, CHECKPOINT_FOLDER, CHECKPOINT_FILE))
    model.save_pretrained(os.path.join(PROJECT_FOLDER, MODEL_FOLDER, str(epoch+1)))
    print("Saving checkpoint at ", os.path.join(PROJECT_FOLDER, CHECKPOINT_FOLDER, CHECKPOINT_FILE))
    print("Saving model at ", os.path.join(PROJECT_FOLDER, MODEL_FOLDER, str(epoch+1)), '\n\n================================================\n')

    return

In [ ]:

# Training without weighted loss
for epoch in range(EPOCHS):
    train(epoch)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Batch 0 of epoch 1 complete.
Training Loss: 1.1147043704986572   Training Accuracy: 34.375
Saving checkpoint at /content/drive/MyDrive/Colab Notebooks/checkpoint.dat
Batch 100 of epoch 1 complete.
Training Loss: 0.8339582268554385   Training Accuracy: 60.51980198019802
Saving checkpoint at /content/drive/MyDrive/Colab Notebooks/checkpoint.dat
Batch 200 of epoch 1 complete.
Training Loss: 0.7307419619750027   Training Accuracy: 66.83768656716418
Saving checkpoint at /content/drive/MyDrive/Colab Notebooks/checkpoint.dat
Batch 300 of epoch 1 complete.
Training Loss: 0.7000133623910505   Training Accuracy: 68.75
Saving checkpoint at /content/drive/MyDrive/Colab Notebooks/checkpoint.dat
Batch 400 of epoch 1 complete.
Training Loss: 0.6706147437678311   Training Accuracy: 70.19170822942644
Saving checkpoint at /content/drive/MyDrive/Colab Notebooks/checkpoint.dat
Batch 500 of epoch 1 complete.
Training Loss: 0.6517189834527151   Training Accuracy: 71.19510978043913
Saving checkpoint at /cont

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



*****

The Total Accuracy for Epoch 1: 72.19047619047619
Training Loss: 0.6316544397690161
Training Accuracy: 72.19047619047619

Validation Loss: 0.5673280091995888
Validation Accuracy: 75.46666666666667

Saving checkpoint at  /content/drive/MyDrive/Colab Notebooks/checkpoint.dat
Saving model at  /Model_V3/1 




In [ ]:

# Evaluation on test set
for epoch in range(1, EPOCHS+1):
  model = BertForSequenceClassification.from_pretrained(os.path.join(PROJECT_FOLDER, MODEL_FOLDER, str(epoch))).cuda()
  print(f'Running validation on model trained on {epoch} epochs')

  validate(model, test_loader)

In [ ]:
model = BertForSequenceClassification.from_pretrained(os.path.join(PROJECT_FOLDER, MODEL_FOLDER, '1')).cuda()
print(f'Running validation on model trained on 1 epochs')

y_true, y_pred, epoch_acc = validate(model, test_loader)

NameError: name 'BertForSequenceClassification' is not defined

In [ ]:

def get_single_prediction(comment, model):
  """
  Predict a star rating from a review comment.

  :comment: the string containing the review comment.
  :model: the model to be used for the prediction.
  """

  df = pd.DataFrame()
  df['text'] = [comment]
  df['rating'] = ['0']

  dataset = ReviewsDataset(df)

  TEST_BATCH_SIZE = 1
  NUM_WORKERS = 1

  test_params = {'batch_size': TEST_BATCH_SIZE,
              'shuffle': True,
              'num_workers': NUM_WORKERS}

  data_loader = DataLoader(dataset, **test_params)

  total_examples = len(df)
  predictions = np.zeros([total_examples], dtype=object)

  for batch, data in enumerate(data_loader):

    # Get the tokenization values.
    input_ids = data['input_ids'].to(device)
    mask = data['attn_mask'].to(device)

    # Make the prediction with the trained model.
    outputs = model(input_ids, mask)

    # Get the star rating.
    big_val, big_idx = torch.max(outputs[0].data, dim=1)
    star_predictions = (big_idx + 1).cpu().numpy()

  return star_predictions[0]


In [ ]:
#Saving the model
torch.save(model.state_dict(), os.path.join(PROJECT_FOLDER, CHECKPOINT_FOLDER, CHECKPOINT_FILE))
model.save_pretrained(os.path.join(PROJECT_FOLDER, MODEL_FOLDER, str(epoch+1)))

In [ ]:
model = BertForSequenceClassification.from_pretrained(os.path.join(PROJECT_FOLDER, MODEL_FOLDER, str(2))).cuda()

In [ ]:
prediction = get_single_prediction("This product is not useful!", model)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
print(prediction)

1


In [ ]:
prediction = get_single_prediction("This product is alright", model)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
print(prediction)

2


In [ ]:
prediction = get_single_prediction("This product is very good!", model)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
print(prediction)

3
